In [ ]:
workdir_ = '/glade/work/juliob/'
import sys
#######################################
# Leave this for now. But it should change to better
# method as here:
# import os
# module_a_dir = os.path.dirname(os.path.abspath(__file__))
# utils_path = os.path.join(module_a_dir, '..', 'Utils')
# sys.path.append(utils_path)
# print( f" a path added in {__name__} {utils_path} ")
########################################
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
#sys.path.append(workdir_ + 'PyRegridding/Regridder/')

# Own local packages
import AveragingUtils as Av
#import VertRegridFlexLL as Vrg  # This is toxic for some reason
import PlotUtil as Pu
import validation_data as Val
import var_A_x_B as vAB
import utils as uti

sys.path.append(workdir_ + 'PyRegridding/Utils/')
import MakePressures as MkP
import GridUtils as GrU


sys.path.append(workdir_ + 'PyRegridding/Drivers/')
import RegridField as RgF


# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( uti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
importlib.reload(vAB)
importlib.reload(MkP)
importlib.reload(RgF)


In [ ]:
####
#   
####
foo='/glade/campaign/cgd/amp/juliob/NOAA_OI_SST/sst.day.mean.2004.nc'

SST=xr.open_dataset(foo)
# Remove February 29, 2004
SST = SST.sel(time=~((SST['time.month'] == 2) & (SST['time.day'] == 29) & (SST['time.year'] == 2004)))

sstnan=SST.sst.values

SST_zer=SST.fillna( 0.)

sstv=SST_zer.sst.values
nt,ny,nx=np.shape( sstv )

sstZon=np.nanmean( sstnan, axis=2 )
sstZonX=np.expand_dims( sstZon, axis=2)
sstZonX= np.repeat( sstZonX, nx , axis=2)

sstYak = np.where(np.isnan(sstnan), sstZonX , sstv)
sstYak = np.where(np.isnan(sstYak), 0. , sstYak)

SST_fix = copy.deepcopy( SST )
SST_fix.sst.values = sstYak


In [ ]:
SST

In [ ]:

ocn_msk = np.where(np.isnan(sstnan), 0. , 1. )
ocn_msk_tsum = np.sum( ocn_msk , axis=0 ) /366.
ocn_msk0 = ocn_msk[0,:,:]

In [ ]:
plt.contourf( ocn_msk0, levels=5 )#np.linspace(-10,40,num=51) )
plt.colorbar()

In [ ]:
from scipy.ndimage import uniform_filter1d

def smooth_sst_ndarray_with_boxcar(sst_array , ocn_msk, iterate=1, size=[3,3] ):
    """
    Smooths the SST field using a boxcar filter, replacing land (fill_value) with smoothed values.
    
    Parameters:
    - sst_array: 3D numpy ndarray containing SST data with dimensions (nt, ny, nx)
    - fill_value: Value representing land (default is -10)
    - size: Size of the boxcar filter window (default is 3x3)

    Returns:
    - sst_smoothed: 3D ndarray with land values replaced by smoothed SST values.
    """
    
    # Step 2: Initialize an array to hold the smoothed results
    sst_smoothed = copy.deepcopy(sst_array) #np.empty_like(sst_array)

    for nit in np.arange( iterate):
        # Loop over time dimension and apply the boxcar filters to each 2D slice
        for t in range(sst_array.shape[0]):  # Loop over the time dimension (nt)
    
            # Step 1: Apply boxcar smoothing along the latitude axis (ny)
            sst_lat_smoothed = uniform_filter1d(sst_smoothed[t], size=size[0], axis=0, mode='reflect')
            
            # Step 2: Apply boxcar smoothing along the longitude axis (nx) with a different boundary treatment
            sst_smoothed[t] = uniform_filter1d(sst_lat_smoothed, size=size[1], axis=1, mode='wrap' ) #, cval=cval_lon)
        
    
        sst_smoothed = np.where( ocn_msk == 1, sst_array , sst_smoothed )
        print( f"Iteration={nit}" )

    return sst_smoothed

# Assuming 'sst_array' is a 3D ndarray with dimensions (nt, ny, nx)
# For example: sst_array = np.random.rand(nt, ny, nx)  # Replace this with your actual data

#sst_smoothed = smooth_sst_ndarray_with_boxcar(sst_array, fill_value=-10, size=3)
#print(sst_smoothed)


In [ ]:
sstYakSm = smooth_sst_ndarray_with_boxcar(sstYak , ocn_msk, iterate=50, size=[17,17] )

In [ ]:
sstYakFx = np.where( ocn_msk == 1, sstYak , sstYakSm )

In [ ]:
plt.contourf( (sstYak)[360,:,:], levels=np.linspace(-10,40,num=51) )

In [ ]:
plt.contourf( (sstYakSm)[360,:,:], levels=np.linspace(-10,40,num=51) )

In [ ]:
plt.contourf( (sstnan)[360,:,:], levels=np.linspace(-10,40,num=51) )

In [ ]:
sstChk = np.where( ocn_msk == 1, sstYakSm , sstnan )

In [ ]:
plt.contourf( (sstChk)[360,:,:], levels=np.linspace(-10,40,num=51) )